In [ ]:
import requests
import pandas as pd
import time
import datetime

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


# Opciones y preferencias de Firefox
options = Options()

# Establece la carpeta de descargas
options.set_preference("browser.download.folderList", 2)  # 0 para descargar en la carpeta de descargas por defecto, 2 para usar una ubicación específica.
options.set_preference("browser.download.dir", ".")  # Cambia "/path/to/download/directory" por tu ruta específica.
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")  # MIME type para pdf
options.set_preference("pdfjs.disabled", True)  # Desactiva el visor de PDF de Firefox
options.headless = True

# Inicia el navegador con las opciones configuradas
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)


In [ ]:
inix = '2016-08-17'
fechas = pd.date_range(start='2017-12-24', end='2023-06-15', freq='D')
fechas = pd.DataFrame(fechas, columns=['f1'])
fechas['f2'] = fechas['f1'].dt.strftime('%d-%m-%Y').astype(str)
fechas['nombre_dia'] = fechas['f1'].dt.day_name().str.upper()
fechas['no_dia'] = fechas['f1'].apply(lambda x: x.weekday())
fechas

In [ ]:
def scrap_page(fecha, nomdia):
    driver.get('https://www.diariooficial.interior.gob.cl/edicionelectronica/index.php?date=' + fecha)
    #esperar hasta que este cargada la imagen
    wait = WebDriverWait(driver, 20)
    element = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/header/a[1]/img')))

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    items = soup.find_all('a', string='Ver edición')
    if len(items) > 0:
        for link in items:
            href = link.get('href')
            driver.get('https://www.diariooficial.interior.gob.cl/edicionelectronica/' + href)
            #esperar hasta que este cargada la imagen
            wait = WebDriverWait(driver, 20)
            element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/header/a[1]/img')))
            element.click()
            time.sleep(0.5)
            get_pdf(fecha)

    else:
        get_pdf(fecha, nomdia)

    return


def get_pdf(fecha, nomdia):
    wait = WebDriverWait(driver, 20)
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/header/div[2]/ul/li[3]/a')))

    url = element.get_attribute('href')
    if url[-5:] == '/.pdf' and nomdia == 'SUNDAY':
        print(fecha, 'Domingo, PDF no disponible')
        return

    elif url[-5:] == '/.pdf':
        print(fecha, 'PDF no disponible')
        return

    element.click()
    time.sleep(2)
    return

In [ ]:
for ix in fechas.index:
    fecha = fechas.loc[ix, 'f2']
    nomdia = fechas.loc[ix, 'nombre_dia']
    try:
        scrap_page(fecha, nomdia)
    except:
        print(fecha, 'Error')
        continue